# Milestone #1: Data Preparation
## Abstract
Music information is an important area in modern information systems, with applications ranging from recommendation engines to lyric-based search platforms. In this project, we present Tunix, a text-based search system inspired by Shazam, but designed to work without audio input. Instead, Tunix allows users to search for songs by typing lyrics, retrieving corresponding tracks, artists, and descriptive metadata.

The final system aims to provide efficient lyric-based search functionality, offering accurate results along with artist descriptions and related attributes, thereby improving music discovery through text queries.

## Introduction
Music retrieval systems such as Shazam have transformed how users interact with music, but they are primarily based on audio fingerprinting. In many cases, users may remember a fragment of lyrics rather than the melody. This project addresses this gap by creating Tunix, a system that retrieves songs, artists, and related information based on text queries derived from lyrics.

Tunix will integrate datasets containing tracks, artists, lyrics, and metadata into a unified and searchable database. The system is designed to return not only the matching track but also relevant contextual information such as artist biography and related metadata. This project contributes to the field of Information Retrieval by developing a structured pipeline for music data collection and preparation, enabling efficient and accurate text-based search functionalities.

## Data search:
- During our search for a suitable music dataset, we found many databases on this topic. However, most of them lacked song lyrics, while others included numerous attributes such as danceability and liveness, which were not relevant to our main objective. The biggest challenge we faced was related to artist information, as it was difficult to find datasets containing proper artist descriptions. To address this issue, we discovered that several APIs provide detailed information: given the name of a song, it is possible to retrieve the artist’s name, description, and additional attributes related to the song, the artist, and even the album.
### Data search Result:
#### search repositories for datasets

| Dataset URL | Attributes | Pros | Cons | Database Type |
|-------------|------------|------|------|---------------|
| [Audio features and lyrics of Spotify songs](https://www.kaggle.com/datasets/imuhammad/audio-features-and-lyrics-of-spotify-songs/data) | track_id, track_name, track_artist, lyrics, track_popularity, track_album_id, track_album_name, track_album_release_date, playlist_name, playlist_id, playlist_genre, playlist_subgenre, danceability, energy, key, loudness, mode, speechiness, instrumentalness, liveness, valence, tempo, duration_ms, language | A lot of information about the songs and many different tracks | Some songs don’t have lyrics and the number of songs per artist is a bit limited. Too many non-essential attributes | Songs |
| [Spotify Million Song Dataset](https://www.kaggle.com/datasets/notshrirang/spotify-million-song-dataset) | song_name, artist_name, link, lyrics | Good balance of attributes, consistent number of songs per artist, and overall a well-sized dataset | Some inconsistencies in how values are obtained — description states data comes from the Spotify API, but song links are from a lyrics website | Songs |
| [Song Lyrics Dataset](https://www.kaggle.com/datasets/deepshah16/song-lyrics-dataset/data) | Artist, Title, Album, Date, Lyric, Year | Well-structured by artist; good complementary dataset to enrich the main dataset with lyrics information | Some songs do not have lyrics (marked consistently with the string "lyrics for this song have yet to be released please check back once the song has been released"); easy to detect and filter | Songs |
| [Lyrics and Metadata from 1950 to 2019](https://data.mendeley.com/datasets/3t9vbwxgr5/2) | artist_name, track_name, release_date, genre, lyrics, len, dating, violence, world/life, night/time, shake the audience, family/gospel, romantic, communication, obscene, music, movement/places, light/visual perceptions, family/spiritual, like/girls, sadness, feelings, danceability, loudness, acousticness, instrumentalness, valence, energy, topic, age | Very rich dataset with a large number of songs and detailed metadata | May have too many artists with only a few songs each and includes many non-essential attributes | Songs |
| [Artists](https://www.upf.edu/web/mtg/semantic-similarity) | artist_name,artist_mbid,biography,top_10_similar_mbids,dbpedia_uri | Very rich dataset with a large number of artists and detailed metadata | Its not a raw dataSet, its a set of files that need to be conected can be harder to work  | Artists |


#### Web scraping / crawling
- Maybe this is the best aprouch since there is not a lot of good datasbases with the artist bio

| Option Name | Method | Pros | Cons |
|-------------|--------|------|------|
| Wikipedia | Web scraping of artist pages | Usually rich and up-to-date textual content; covers a wide range of artists; freely accessible | Needs to handle disambiguation of artist names; scraping may be blocked if too many requests; text may require cleaning |
| DBpedia | SPARQL queries on DBpedia endpoint | Structured data extracted from Wikipedia; supports multiple languages; can get abstracts directly | Limited to artists with DBpedia entries; not all biographies are complete; requires knowledge of SPARQL |
| Wikidata | SPARQL queries on Wikidata endpoint | Very structured; easy to link with other datasets via QIDs; can extract multiple attributes | Not all artists have detailed biographies; abstracts are shorter than Wikipedia; requires SPARQL knowledge |
| Last.fm API | Queries the `artist.getInfo` endpoint to get `bio.content` | Easy to use; provides textual content ready for NLP | Low rate limiting (but manageable if calls are distributed) |
| Genius API | Queries the `/search` endpoint and retrieves the artist page `/artists/:id` | Focused on artist bios; very rich content | Low rate limiting; requires an API key |


## Select convenient data subsets

- Despite the availability of several good databases, we chose the Spotify Million Song Dataset. Although it contains fewer features, it is rich in music records and has fewer distinct artists, which reduces the number of required API calls. This way, we obtain fewer artists but more songs per artist. We also assumed that any additional attributes could later be retrieved through APIs to complement the dataset with artist information. 
- For this purpose, we selected the Genius API, as it focuses specifically on artists and music, providing detailed descriptions and additional metadata.

## Assess the authority of the data source and data quality
- Dataset: Spotify Million Song Dataset – Kaggle:
    - Authority: Public dataset on Kaggle, licensed CC0 (free to use). It’s a subset of the original Million Song Dataset.
    - Data Quality: Contains song names, artist names, lyrics, and links. Some entries may be missing or duplicate. 
- Genius API:
    - Authority: Genius is a well-known platform for song lyrics and artist info. The API is official, documented, and widely used.
    - Data Quality: Provides structured, regularly updated data. Some songs or artists may be missing, and user-contributed info can vary in quality.
## Exploratory Data Analysis:

In [1]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn


  Using cached numpy-2.3.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 2.8 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.0 MB 3.4 MB/s eta 0:00:03
   ------- -------------------------------- 2.1/11.0 MB 3.8 MB/s eta 0:00:03
   ----------- ---------------------------- 3.1/11.0 MB 4.2 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/11.0 MB 4.6 MB/s eta 0:00:02
   --------------------- ------------------ 6.0/11.0 MB 5.1 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.0 MB 5.5 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.0 MB 5.9 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 6.3 MB/s eta 0:00:01
   ------------------------------


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached matplotlib-3.10.6-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-11.3.0-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.6-cp312-cp312-win_amd64.whl (8.1 MB)
Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl (226 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   --------------------------- ------------ 1.6/2.3 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 8.0 MB/s eta 0:00:00
Using cached kiwisolver-1.4.9-cp312-cp312-win_amd64.whl (73 kB)
Using cached pillow-11.3.0-cp312-cp312-win_amd64.whl (7.0 MB)
Using cached pyparsing-3.2.5-py3-none-an


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import data_utils
import importlib

importlib.reload(data_utils)
df=data_utils.load_data()
data_utils.dataAnalysis(df)
#data_utils.plot_artist_frequency(df, col="artist", top_n=20,most=True)
#data_utils.plot_artist_frequency(df, col="artist", top_n=20,most=False)
data_utils.plot_column_frequency(df,"song",20)


data loaded
===== DATAFRAME SHAPE =====
Rows: 57650, Columns: 4

===== COLUMN NAMES AND DATA TYPES =====
artist    object
song      object
link      object
text      object
dtype: object 

===== MISSING VALUES =====
No missing values

===== DUPLICATE ROWS =====
Number of duplicate rows: 0

===== UNIQUE VALUES PER COLUMN =====
artist: 643 unique values
song: 44824 unique values
link: 57650 unique values
text: 57494 unique values

===== Average song per artis =====
89.65785381026438

===== BASIC STATISTICS FOR NUMERIC COLUMNS =====
No numeric columns.

===== MOST FREQUENT VALUES FOR CATEGORICAL COLUMNS =====
artist:
artist
Donna Summer        191
Gordon Lightfoot    189
Bob Dylan           188
George Strait       188
Loretta Lynn        187
Name: count, dtype: int64

song:
song
Have Yourself A Merry Little Christmas    35
Angel                                     28
Home                                      27
Hold On                                   27
I Believe                        

### Exploratory Data Analysis resume:
- **Atributes**:
    - artist-Artist Name
    - Song- The name of the song
    - Link-An identifier of the song
    - Text- The lyric of the song
- **Lyrics Structure**
    - Paragraphs separated by `\r\n` (each line or group of lines is a verse or pause)
    - Repeated lines indicate choruses or refrains
    - Variable length (short or long lyrics)
    - Keywords may indicate themes (e.g., "Christmas", "love", "baby")
    - Some lines contain musical notes or instructions (e.g., "[Chorus]")
    - Formatting may vary slightly between songs (extra spaces, capitalization)
    - Identical lyrics can appear for different artists (covers)

- **Link structure**
  - Each link points to a **unique song page** on lyricsfreak
  - The URL usually contains:
    - **Artist name** (`/a/abba/` → ABBA)
    - **Song title** (`ahes+my+kind+of+girl`)
  - The number at the end (e.g., `_20598417`) is a **unique  song ID**

- **Unique values**:
    - Looking to the unique values we can conclude that there in mean like 89 songs per artis which is your goal since we dont want to call the artist APi that much
    - The low unique values of the songs is because there are a lot of songs with the same name and  difrent version of the same song with difrent singers
    - The diference between the number of links and the number of text is because we have songs with same lyrics they are the same songs but they have difrent artist singing it 
    


## Prepare and Document a Data Processing Pipeline

### Main Idea
The goal is to **enrich our dataset** with additional information obtained via the **Genius API**.  
We want to collect:
- **Primary Artist Description**
- **Album Description**

### How to Achieve This
1. **Iterate through dataset entries** (songs and artists).
2. **Search endpoint**: Use the Genius API search to retrieve the Genius IDs for both the **artist** and the **song**.  
   - Endpoint:  
     ```
     GET https://api.genius.com/search?q={song name} {artist name}
     ```
3. **Retrieve album information** using the song ID.  
   - Endpoint:  
     ```
     GET https://api.genius.com/songs/{SONG_ID}
     ```
   - Data retrieved: Album name, URL, cover art, release date (if available).
4. **Retrieve artist information** using the artist ID.  
   - Endpoint:  
     ```
     GET https://api.genius.com/artists/{ARTIST_ID}
     ```
   - Data retrieved: Artist description, bio, and additional metadata.
### Data storage:
To improve data organization and scalability, we will structure the database into **three main tables** (with the possibility of adding more attributes if needed):

#### 1. Musics
- **Music_ID** (primary key)  
- **Artist_ID** (foreign key → Artist table)  
- **Album_ID** (foreign key → Album table)  
- **Lyrics**  

#### 2. Artists
- **Artist_ID** (primary key)  
- **Artist_Name**  
- **Artist_Bio**  

#### 3. Albums
- **Album_ID** (primary key)  
- **Album_Name**  
- **Description**  

### Expected Outcome
- Enriched dataset that contains:
  - Original song data (from our CSV)
  - Linked album metadata (title, release date, cover art, URL)
  - Linked artist metadata (biography/description)
- Improved analysis possibilities (e.g., comparing songs across albums, artist popularity, thematic lyric analysis).

